In [ ]:
import csv
import sys
import io
import json
import re
from collections import Counter

In [2]:
%%time
csv.field_size_limit(99999999)
vacs = list(csv.DictReader(io.open('Vacancy.csv', encoding='utf-8')))

Wall time: 3.09 s


### если потребуется вывести вакансию в html

In [ ]:
decode_html = json.loads(vacs[0]['vachtml'])
decode_html

# Сколько вакансий, которые вам нравятся?
Допустим, мне нравятся все вакансии, у которых в описании есть слова sql и python

Добавим в каждый словарь ключ/признак, нравится ли мне эта вакансия (true) или нет (false)

In [4]:
%%time
for i in range(len(vacs)):
    #добавим в словарь признак, нравится ли мне вакансия
    vacs[i]['is_good_vac'] = ('python' in vacs[i]['vacdescription'].lower()) or ('sql' in vacs[i]['vacdescription'].lower())
    
    #удалим из словаря код html, чтобы не мешался
    vacs[i].pop('vachtml', None)

Wall time: 609 ms


In [5]:
vacs[0]

OrderedDict([('id', '383479'),
             ('created_at', '2021-03-14 09:39:21.854462'),
             ('updated_at', '2021-03-14 09:39:21.854584'),
             ('vacid', '1474941'),
             ('vactitle', 'Senior java-разработчик (в команду Онбординга)'),
             ('vacdescription',
              ' Работа у нас — это:команда профессионалов, готовых поддержать ваши инициативы;мощное железо, дополнительные мониторы и всё, что нужно для продуктивной работы;обучение за счёт компании;гибкий графиккрасивый и комфортный офис  г. Санкт-Петербург, м. Ладожская  пл. Карла Фаберже 8 ВДМС;прозрачная система бонусов и премий, достойная зарплата — размер обсудим на собеседовании.Информация о проекте:Наша команда делает новые экспериментальные фичи для СБОЛа.Сейчас мы разрабатываем систему, с помощью которой разные команды Сбера смогут создавать квесты для мобильного приложения.Также мы будем разрабатывать внутреннюю платформу для продуктовых экспериментов. Описание команды:2-pizza team: сей

посчитаем все вакансии, в которых признак is_good_vac == true


In [6]:
sum([x['is_good_vac'] for x in vacs])

1289

# Насколько свежие эти вакансии?

Создадим список, в котором будет указана дата вакансии, если она мне нравится. Если она мне не нравится, что будет указана None

In [7]:
my_vacs_date = [x['vacdate'] if x['is_good_vac'] else None for x in vacs]
my_vacs_date[:5]

['2021-03-15', None, None, '2021-03-15', '2021-03-15']

отфильтруем список

In [8]:
my_vacs_date = list(filter(lambda x: x != None, my_vacs_date))
my_vacs_date[:5]

['2021-03-15', '2021-03-15', '2021-03-15', '2021-03-15', '2021-03-15']

Выведем наиболее встречаемые даты вакансий

In [9]:
Counter(my_vacs_date).most_common(20)

[('2021-03-16', 59),
 ('2021-03-17', 58),
 ('2021-03-15', 56),
 ('2021-03-01', 55),
 ('2021-03-03', 53),
 ('2021-03-02', 47),
 ('2021-03-11', 46),
 ('2021-03-10', 46),
 ('2021-02-24', 42),
 ('2021-03-09', 41),
 ('2021-03-05', 37),
 ('2021-02-25', 36),
 ('2021-02-17', 34),
 ('2021-02-03', 32),
 ('2021-03-12', 31),
 ('2021-03-04', 30),
 ('2021-02-16', 28),
 ('2021-01-11', 26),
 ('2021-02-26', 25),
 ('2021-02-19', 25)]

Видим, что большинство вакансий от февраля/марта.

Проверим какой у вакансий статус

In [10]:
my_vacs_status = [x['vacstatus'] if x['is_good_vac'] else None for x in vacs]
my_vacs_status = list(filter(lambda x: x != None, my_vacs_status))
Counter(my_vacs_status).most_common()

[('new', 1289)]

У всех статус new

# Сколько вакансий с позициями на которых вы работаете?

Попробуем найти вакансии в которых будет указана фраза сопровождение транзакционного...

In [11]:
%%time
for i in range(len(vacs)):
    #добавим в словарь признак
    vacs[i]['similar_vac'] = re.findall(r'\bсопровожден.. \bтранз', vacs[i]['vacdescription'].lower()) != []

Wall time: 954 ms


In [12]:
sum([x['similar_vac'] for x in vacs])

0

Таких оказалось 0

# Сколько вакансий для аналатика данных?
Попробуем найти вакансии, у которых в названии есть слово "аналитик"

In [13]:
%%time
pattern = r'data.anal|data.аналит|аналитик.данн|аналитик.dat'
for i in range(len(vacs)):
    #добавим в словарь признак
    vacs[i]['vac_for_analytic'] = re.findall(pattern, vacs[i]['vacdescription'].lower()) != []

Wall time: 565 ms


In [14]:
sum([x['vac_for_analytic'] for x in vacs])

21

Таких оказалось 21

# Сколько вакансий для аналитика данных с использованием Python?
Найдем вакансии,  у которых в названии есть слово "аналитик" и дополнительно в описании есть слово "python"

In [15]:
%%time
pattern = r'data.anal|data.аналит|аналитик.данн|аналитик.dat'
for i in range(len(vacs)):
    #добавим в словарь признак
    vacs[i]['vac_for_analytic_with_python'] = (re.findall(pattern, vacs[i]['vacdescription'].lower()) != []) and ('python' in vacs[i]['vacdescription'].lower())

Wall time: 518 ms


In [16]:
sum([x['vac_for_analytic_with_python'] for x in vacs])

12

Таких вакансий оказалось 12